In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
pd.set_option('display.max_rows', 10)

Для примера, возьмем маленький DataFrame из 6 строк, где поля col_1 и col_2 некий набор данных в которых мы хотим найти пары (45, 4), (45, 5), (56, 4) и (56, 5). Целевая переменная target. 

In [2]:
df = pd.DataFrame({'col_1':[45, 56, 67, 78, 45, 56],
                   'col_2':[4, 9, 6, 7, 5, 4],
                   'target':[1, 0, 0, 0, 1, 1]})

In [3]:
df.head(10)

,col_1,col_2,target
0,45,4,1
1,56,9,0
2,67,6,0
3,78,7,0
4,45,5,1
5,56,4,1


In [4]:
features = ['col_1','col_2']
X = df[features].values
y = df.target.values

In [5]:
logit = LogisticRegression(C=1, random_state=42, solver='liblinear')

In [6]:
logit.fit(X, y)

LogisticRegression(C=1, random_state=42, solver='liblinear')

In [7]:
train_pred = logit.predict_proba(X)[:, 1]

In [8]:
roc_auc_score(y, train_pred)

0.7777777777777778

Точность нашей модели получилась не маленькая, давайте проверим:

In [9]:
df['pred'] = train_pred
df.head(10)

,col_1,col_2,target,pred
0,45,4,1,0.511164
1,56,9,0,0.143420
2,67,6,0,0.454502
3,78,7,0,0.426517
4,45,5,1,0.404945
5,56,4,1,0.589197


In [10]:
logit.predict_proba([[56, 1],[56, 5],[56, 4]])[:, 1]

array([0.838801  , 0.48277738, 0.5891966 ])

Наша модель что-то предсказывает, но совсем не то, что нам нужно.
Разложим наши данные на бинарные признаки (закодируем наши пары) и заново обучим нашу модель.

In [11]:
df = pd.get_dummies(df, columns=['col_1', 'col_2'])
df.head(10)

,target,pred,col_1_45,col_1_56,col_1_67,col_1_78,col_2_4,col_2_5,col_2_6,col_2_7,col_2_9
0,1,0.511164,1,0,0,0,1,0,0,0,0
1,0,0.143420,0,1,0,0,0,0,0,0,1
2,0,0.454502,0,0,1,0,0,0,1,0,0
3,0,0.426517,0,0,0,1,0,0,0,1,0
4,1,0.404945,1,0,0,0,0,1,0,0,0
5,1,0.589197,0,1,0,0,1,0,0,0,0


In [12]:
X = df.drop(columns=['target', 'pred']).values

In [13]:
logit.fit(X, y)
train_pred = logit.predict_proba(X)[:, 1]
roc_auc_score(y, train_pred)

1.0

Итак, точность нашей модели по метрике roc-auc стала максимальной, проверим:

In [14]:
df['pred'] = train_pred
df.head(10)

,target,pred,col_1_45,col_1_56,col_1_67,col_1_78,col_2_4,col_2_5,col_2_6,col_2_7,col_2_9
0,1,0.749328,1,0,0,0,1,0,0,0,0
1,0,0.381602,0,1,0,0,0,0,0,0,1
2,0,0.323312,0,0,1,0,0,0,1,0,0
3,0,0.323312,0,0,0,1,0,0,0,1,0
4,1,0.686790,1,0,0,0,0,1,0,0,0
5,1,0.627597,0,1,0,0,1,0,0,0,0


Проверим пары (56, 5) и (56, 7)

In [15]:
logit.predict_proba([[0, 1, 0, 0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0, 1, 0, 0]])[:, 1]

array([0.5528129 , 0.39544499])

### Интересующая нас пара (56, 5) в обучении не участвовала, но наша модель смогла ее определить и дать вероятность больше 50% 